# Train a Scikit-Learn model in SageMaker and track with MLFlow

## Setup Environment

In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker

In [2]:
!pip install --upgrade numpy scikit-learn

  Using cached numpy-2.2.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.11 requires mkl, which is not installed.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.2.4 which is incompatible.
sagemaker 2.243.0 requires numpy<2.0,>=1.9.0, but you have numpy 2.2.4 which is incompatible.


In [3]:
!pip install numpy==1.26.4

!pip install --upgrade pandas


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.11 requires mkl, which is not installed.


In [4]:
import sagemaker
import pandas as pd
# from sklearn.datasets import load_boston
from sagemaker.sklearn.estimator import SKLearn
# from sklearn.model_selection import train_test_split

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()

# uri of your remote mlflow server
tracking_uri = 'http://MLflow-MLFLO-VR3wJBi9L1nl-09870ac901267fb3.elb.us-east-1.amazonaws.com/' 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/29/25 09:17:13] INFO     Found credentials from IAM Role:                                   ]8;id=974603;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=501124;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=553545;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=934818;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/29/25 09:17:14] INFO     Found credentials from IAM Role:                                   ]8;id=44134;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=273658;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Prepare data
We load a dataset from sklearn, split it and send it to S3

In [9]:
# we use the Boston housing dataset 
# data = load_boston()

# X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.25, random_state=42)

# trainX = pd.DataFrame(X_train, columns=data.feature_names)
# trainX['target'] = y_train

# testX = pd.DataFrame(X_test, columns=data.feature_names)
# testX['target'] = y_test

# trainX.to_csv('boston_train.csv')
# testX.to_csv('boston_test.csv')

In [5]:
# send data to S3. SageMaker will take training data from s3
train_path = sess.upload_data(path='boston_train.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')
test_path = sess.upload_data(path='boston_test.csv', bucket=bucket, key_prefix='sagemaker/sklearncontainer')

## Train

In [6]:
hyperparameters = {
    'tracking_uri': tracking_uri,
    'experiment_name': 'boston-housing',
    'n-estimators': 200,
    'min-samples-leaf': 2,
    'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
    'target': 'target'
}

metric_definitions = [{'Name': 'median-AE', 'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}]

estimator = SKLearn(
    entry_point='train.py',
    source_dir='source_dir',
    role=role,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.0-1',
    base_job_name='mlflow',
)

[03/29/25 09:17:28] INFO     Found credentials from IAM Role:                                   ]8;id=110676;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=192439;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/29/25 09:17:29] INFO     Found credentials from IAM Role:                                   ]8;id=371988;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=32766;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [7]:
estimator.fit({'train':train_path, 'test': test_path})

[03/29/25 09:17:32] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=97621;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=294406;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[03/29/25 09:17:33] INFO     Creating training-job with name: mlflow-2025-03-29-09-17-32-772        ]8;id=805010;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=965941;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-29 09:17:34 Starting - Starting the training job...
2025-03-29 09:17:48 Starting - Preparing the instances for training...
2025-03-29 09:18:29 Downloading - Downloading the training image......
2025-03-29 09:19:25 Training - Training image download completed. Training in progress..2025-03-29 09:19:35,351 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-03-29 09:19:35,355 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-29 09:19:35,358 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-29 09:19:35,373 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-03-29 09:19:35,645 sagemaker-training-toolkit INFO     Installing module with the following command:
/miniconda3/bin/python -m pip install . -r requirements.txt
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with s

2025-03-29 09:19:46,287 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-29 09:19:46,290 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-29 09:19:46,309 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-29 09:19:46,312 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-29 09:19:46,331 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-29 09:19:46,334 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-29 09:19:46,349 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluste


2025-03-29 09:20:03 Uploading - Uploading generated training model
2025-03-29 09:20:03 Completed - Training job completed
Training seconds: 114
Billable seconds: 114
